In [ ]:
# kernel Image_pytorch_2.1.0-cuda11.8-cudnn8-devel

In [ ]:
# 套件目錄
!ls ~/.local/lib

In [ ]:
# 新增可讀取套件安裝位置
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin'
os.environ['PATH']=Add_Binarry_Path+':'+os.environ['PATH']

In [ ]:
# 安裝套件
!pip install xformers bitsandbytes sentence_transformers einops arxiv openai chromadb tiktoken pymupdf langchain chainlit transformers IProgress ipywidgets accelerate -q

In [ ]:
# 安裝套件 GPTQ
!pip install langchain optimum  transformers IProgress ipywidgets accelerate -q
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  -q # Use cu117 if on CUDA 11.7

In [ ]:
from langchain.llms import HuggingFacePipeline
# GPTQ
from transformers import AutoModelForCausalLM, pipeline, AutoTokenizer, TextStreamer
import transformers
import torch

In [ ]:
def load_llama():
    MODEL_ID = "/work/u00cjz00/slurm_jobs/github/models/Llama-2-7B-Chat-GPTQ"
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        max_length=1000,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
        #use_auth_token=True,
    )

    llm = HuggingFacePipeline(
        pipeline=pipeline,
        model_kwargs={"temperature": 0},
    )
    return llm


llm = load_llama()

In [ ]:
llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")